In [1]:
import pandas as pd

# Estimate daily concentrations of key chemical species

Aim: Derive daily time series of concentrations of the key variables of interest to the marine folk.

Time period of interest to be determined. Storm Hans is of particular interest, but we may be interested all the way back to 1990. **Discuss with Helene**

Do this using either:
1) Regressions with sensor data data (for 2023 only and Glomma only) (derived in notebook flux1_regressions), if regressions were meaningful
2) Regressions with discharge data (also derived in notebook flux1_regressions)
3) Interpolation of grab sample data if data frequency appears high enough
4) Filling using e.g. monthly median values if none of the above are appropriate

Output of this notebook is then used to estimate fluxes to the fjord.